In [ ]:
import spacy
import pandas as pd
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy.tokens import Doc
from spacy.util import minibatch, compounding
import nltk
nltk.download('vader_lexicon')
spacy.require_gpu()
#nlp  = spacy.load('en_core_web_sm')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

Hear we are using spacy en_core_web_lg model which is spacy's full version for english language, we are using this model with google col;ab GPU's to get better and fast results.

In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180944 sha256=c86f56477a3299ab9f33afad06393ddcbc4e0a4868eace910a8d2e3d2b44d9fa
  Stored in directory: /tmp/pip-ephem-wheel-cache-anmgjtzl/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:

#laoding the model 
nlp  = spacy.load('en_core_web_lg')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/StrawHats projects/News scraping.csv')
data = data.drop(['Language','Date','Time','Source Link','Author/Publisher'], axis=1)
data.head()

,Category,Sub_Category,Title,Synopsis,News
0,Sports,Badminton,BWF World Tour Finals: Fighting PV Sindhu lose...,This was PV Sindhu's 16th defeat to Tai Tzu Yi...,World champion shuttler P V Sindhu went down f...
1,Sports,Badminton,"World Tour Finals Preview: PV Sindhu, recharge...",With the Indian having played more matches tha...,Carolina Marin (50 total) played 24 tournament...
2,Sports,Badminton,Satwiksairaj’s offence gets neutralised by sav...,Satwiksairaj Rankireddy uses big smash to kill...,One would have to be blind to not figure that ...
3,Sports,Badminton,Dream run of Indian doubles pairs end with sem...,"Up against the world number three Thai pair, S...",The Indian mixed doubles pair of Satwiksairaj ...
4,Sports,Badminton,Satwik-Chirag’s impressive run ends with semif...,The Indian pair had participated in Super 1000...,Tokyo Olympics medal contender Satwiksairaj Ra...


#NER 

We are using the Named entity recognition(NER) technique to process and analyse all the web scraped data. \\

**Named entity recognition(NER):-** NER represents the technique of chunking, extraction or identification of the particular entities in the data. It helps to identify and categorize critical information in the text. An entity can be any word or series of words that consistently refers to the same thing. Each newly detected entity is classified into a predetermined category.

In [ ]:
import spacy
from wordcloud import STOPWORDS, WordCloud
from spacy.util import minibatch,compounding


These two codes will extract entities from the news as tags and we will use those tags for training our LSH modelfor accurate results.

In [ ]:
def NER_tags(in_news):
  tag = []

  for text in in_news:
    text = str(text)
    doc = nlp(text)
    for ent in doc.ents:
      if ent.label_ == 'CARDINAL':
        pass
      elif ent.label_ == 'ORDINAL':
        pass
      elif ent.label_ == 'DATE':
        pass
      elif ent.label_ == 'TIME':
        pass
      else:
        tag.append(ent.text)
        tag_in = ' '.join(tag)
  return tag_in

def hash_tags(in_news):
    tag = []

  
    text = str(in_news)
    doc = nlp(text)
    for ent in doc.ents:
      if ent.label_ == 'CARDINAL':
        pass
      elif ent.label_ == 'ORDINAL':
        pass
      elif ent.label_ == 'DATE':
        pass
      elif ent.label_ == 'TIME':
        pass
      else:
        tag.append(ent.text)
        tag_in = ' '.join(tag)
    return tag_in 



Hear we are using above functions to extract tags from the news contents so we are not going to miss any importent detail or tag related to news, and these NER tags are extremly useful in our NER+LSH algorithm which is giving us very good results

In [ ]:
data['tags'] = data['News'].apply(lambda x: hash_tags(x))


In [ ]:
data

,Category,Sub_Category,Title,Synopsis,News,tags
0,Sports,Badminton,BWF World Tour Finals: Fighting PV Sindhu lose...,This was PV Sindhu's 16th defeat to Tai Tzu Yi...,World champion shuttler P V Sindhu went down f...,Tai Tzu Ying Taiwan $1.5 million HSBC BWF Worl...
1,Sports,Badminton,"World Tour Finals Preview: PV Sindhu, recharge...",With the Indian having played more matches tha...,Carolina Marin (50 total) played 24 tournament...,Carolina Marin PV Sindhu Indian Marin Sindhu O...
2,Sports,Badminton,Satwiksairaj’s offence gets neutralised by sav...,Satwiksairaj Rankireddy uses big smash to kill...,One would have to be blind to not figure that ...,Satwiksairaj Rankireddy Neutralising Malaysian...
3,Sports,Badminton,Dream run of Indian doubles pairs end with sem...,"Up against the world number three Thai pair, S...",The Indian mixed doubles pair of Satwiksairaj ...,Indian Satwiksairaj Rankireddy Ashwini Ponnapp...
4,Sports,Badminton,Satwik-Chirag’s impressive run ends with semif...,The Indian pair had participated in Super 1000...,Tokyo Olympics medal contender Satwiksairaj Ra...,Tokyo Olympics Satwiksairaj Rankireddy Chirag ...
...,...,...,...,...,...,...
5918,Entertainment,Box-office-collection,Kesari box office collection Day 8: Akshay Kum...,Kesari box office collection Day 8: Akshay Kum...,"Kesari, starring Akshay Kumar in the lead role...",Kesari Akshay Kumar Rs 105.86 crore Taran Adar...
5919,Entertainment,Box-office-collection,Junglee box office prediction: Vidyut Jammwal ...,Junglee box office prediction: Junglee will ha...,Vidyut Jammwal’s Junglee has caught the fancy ...,Vidyut Jammwal’s Junglee Kesari Notebook Chuck...
5920,Entertainment,Box-office-collection,Luka Chuppi box office collection Day 26: Kart...,Luka Chuppi box office collection Day 26: Krit...,Kriti Sanon and Kartik Aaryan’s film Luka Chup...,Kriti Sanon Kartik Aaryan Luka Chuppi Rs 43 la...
5921,Entertainment,Box-office-collection,Badla box office collection day 20: Going gets...,Badla box office collection day 20: Taapsee Pa...,"After more than 2 weeks, Taapsee Pannu’s film ...",Taapsee Pannu’s Badla Badla Rs 1 crore only Rs...


In [ ]:
tag =  NER_tags(data['Synopsis'][234:237])
tag

'Satwik the World the Fuzhou China Open 2019 Indian Marcus Kevin the BWF World Tour Super Satwik -Chirag Marcus-Kevin Fuzhou Indian Marcus Fernaldi Gideon Kevin Sanjaya French Open'

In [ ]:
hash_tag = hash_tags(data['Synopsis'][234:237])
ta = list(set(hash_tag))
hash_tag

'Satwik-Chirag Indian Satwik-Chirag Marcus-Kevin Fuzhou'

#LSH


We are using the technique of Locality-sensitive hashing(LSH) to find out the most relevant news articles to recommend to a user. \\
**Locality-sensitive hashing(LSH)**:- LSH is a model with the help of which we can sort out the similar input items into the same bucket. Here the number of buckets is small compared to the number of input items. So, we can sort out mostly similar items together in the same bucket and present them in hashtags. \\

To understand the LSH technique mathematically, let us define a Hash family $H$. \\

Now, $P[h(x) = h(y)]$ indicates the probability such that two points $h(x)$ and $h(y)$ in the Hash family H are equal. The Hash family $H$ is locality sensitive if, \\

$P[h(x) = h(y)]$  is high if $x$ is close to $y$   , \\

$P[h(x) = h(y)]$  is low if $x$ is away from $y$. \\

The high probabilty indicates that the two points are likely to be included in the same bucket. \\

The similarity between two points is calculated using the known method of Jaccard similarity. The Jaccard Similarity index is calculated as, \\

$J(A,B) = \frac{|A \cap B|}{|A \cup B |}$ 

Where, $J(A,B)$ is the Jaccard distance between two sets points A and B. 

Mathematically, we can write ,

For a Given universe $U$ and similarity $s: U \times U \rightarrow [0,1]$ , There exists a probability distribution over some Hash family H such that 

$P[h(x) = h(y)] = s(x,y)$ 

whre $ h \in H$ and $s(x,y) = s(y,x)$. 

And $s(x,y) = 1 \rightarrow x=y $

We are going to use data scatch library to make LSH model by usinfg MinHash forest you can see the details heaar  https://github.com/ekzhu/datasketch

In [ ]:
!pip install datasketch

     |████████████████████████████████| 71kB 6.8MB/s 


In [ ]:
import datasketch
from datasketch import MinHash , MinHashLSH, MinHashLSHEnsemble, MinHashLSHForest
import re
import string
import time
import pandas as pd
import numpy as np
from spacy.lang.en.stop_words import STOP_WORDS
stop_words = STOP_WORDS

Hear are all the text pre-processing that we are using in our algorithm

In [ ]:

def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 
    
    return text2.lower()

#Cleantext for News
data['News'] = data['News'].apply(clean_text)
#cleantext for synopsis
data['Synopsis'] = data['Synopsis'].apply(clean_text)



In [ ]:
#Lets remove the common words
# Common word removal
common_words = pd.Series(' '.join(data['News']).split()).value_counts()[:50]
common_words = list(common_words.index)
data['News'] = data['News'].apply(lambda x: " ".join(x for x in x.split() if x not in common_words))
#common wordes removel for synopsis
common_words = pd.Series(' '.join(data['Synopsis']).split()).value_counts()[:50]
common_words = list(common_words.index)
data['Synopsis'] = data['Synopsis'].apply(lambda x: " ".join(x for x in x.split() if x not in common_words))

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the News
data['news_without_stop']=data['News'].apply(remove_stopwords)



nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

text_list=data['news_without_stop'].tolist()
print(text_list[1])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[1])

dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]


In [ ]:
#Preprocess will split a string of text into individual tokens/shingles based on whitespace.
def preprocess(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    text = re.sub(r'[^\w\s]','',rem_text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

Hear what the main LSH work is going on so what we are doing hear is we are using MinhashLSH forest to make our model based on NER tags that we have extracted eirlier

In [ ]:
#data['tokenized'] = data['Synopsis'].apply(preprocess)
#Number of Permutations
permutations = 128

#Number of Recommendations to return
num_recommendations = 1

def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for text in data['tags']:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = database.iloc[idx_array]['News']
    ls_idx = idx_array.tolist()
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return ls_idx , result


In [ ]:
 #index to data frame function
 def idx_to_df(idx, data):
   c = (data.iloc[idx]).to_frame().T
   return c
idx_to_df(245 , data)

,Category,Sub_Category,Title,Synopsis,News,tags
245,Sports,Badminton,Lakshya Sen beats China’s Weng Hong Yang to cl...,Lakshya Sen bagged his second successive BWF W...,India’s Lakshya Sen bagged his second successi...,India Lakshya Sen BWF World Tour Super 100 Chi...


In [ ]:

forest = get_forest(data, permutations)

It took 11.715883731842041 seconds to build forest.


First test

In [ ]:
num_recommendations = 10
Synopsis = tag # tag is defined above in NER section
result = predict(Synopsis, data, permutations, num_recommendations, forest)[1]
print('\n Top Recommendation(s) is(are) \n', result)


It took 0.013955116271972656 seconds to query forest.

 Top Recommendation(s) is(are) 
 258     French Open 2019 Live Streaming: Saina Nehwal,...
2341    WWE superstar Nia Jax created history on Monda...
261     Kuhoo Garg and Dhruv Rawat put up a brilliant ...
296     China Open Badminton 2019 Live Score Streaming...
297     China Open Badminton 2019 Schedule, Players Li...
236     Satwik-Chirag vs Marcus-Kevin, Fuzhou China Op...
273     Denmark Open Badminton 2019 Schedule, Players ...
2323    WWE Elimination Chamber 2019: On the road to W...
2295    WWE Fastlane 2019 in Cleveland, Ohio was the p...
4863    David Lynch should ask Remedy Entertainment fo...
Name: News, dtype: object


So our model is working perfectly so lets make a recommander which will use user data to give recommandations

In [ ]:
def recommander(News_in , num_reco):
  results = pd.DataFrame(columns= ['Category', 'Sub_Category', 'Title',
       'Synopsis', 'News'])
  
  num_recommendations =num_reco
  permutations = 128
  NER_tag = NER_tags(News_in)
  result = list(predict(NER_tag, data, permutations, num_recommendations, forest))[0]
  for idx in result:
      c = idx_to_df(idx , data)
      results = results.append(c)
  return results




right now we dont have user data so we will use existing data so this is from sport section football and as we can see the recommandations we are getting some good results

In [ ]:
recommander(data['tags'][789:800] , 10)

It took 0.007288932800292969 seconds to query forest.


,Category,Sub_Category,Title,Synopsis,News,tags
299,Sports,Badminton,PV Sindhu eyes China Open after World Champion...,PV Sindhu ended India's long wait for a world ...,World champion PV Sindhu will look to reassert...,Indian China Changzhou Sindhu India Basel Swit...
719,Sports,Football,Mount delivers again for Lampard as Chelsea ek...,Four months into the season and Chelsea manage...,Four months into the season and Chelsea manage...,Chelsea Frank Lampard $300 million Mason Mount...
881,Sports,Football,Lionel Messi’s salary at Barcelona unsustainab...,"Lionel Messi, who sought an exit from Barcelon...",Lionel Messi’s salary is too big for Barcelona...,Lionel Messi Barcelona Emili Rousaud Messi Bar...
252,Sports,Badminton,French Open: Satwik/Chirag continue golden run...,"French Open: Saina Nehwal suffered a 20-22, 21...",Satwiksairaj Rankireddy and Chirag Shetty stor...,Satwiksairaj Rankireddy Chirag Shetty the Fren...
789,Sports,Football,Lionel Messi returns for Barcelona as Koeman w...,Lionel Messi has stated he will not decide his...,Lionel Messi has recovered from an ankle injur...,Lionel Messi Huesca Barcelona La Liga Eibar Ro...
247,Sports,Badminton,BWF Rankings: Satwik-Chirag reclaim top-10 spo...,PV Sindhu and Saina Nehwal remained static at ...,Indian men’s doubles pairing of Satwiksairaj R...,Indian Satwiksairaj Rankireddy Chirag Shetty B...
984,Sports,Football,Lionel Messi pays tribute to Diego Maradona in...,"After scoring Barcelona's fourth goal, Lionel ...",Lionel Messi paid a personal tribute to the la...,Lionel Messi Diego Maradona Barcelona Osasuna ...
892,Sports,Football,Lionel Messi fires anxious Barcelona to victor...,The win lifted Barcelona up to eighth in the s...,Lionel Messi dragged a nervous Barcelona to a ...,Lionel Messi Barcelona Levante La Liga Catalan...
797,Sports,Football,"As transfer window opens, struggles of Premier...","With the January transfer window opening, Prem...",The high expectations from new recruits aren’t...,Tottenham Bale Real Madrid Bale Madrid Harry K...
2206,Sports,Wwe-wrestling,"WWE Raw Results: The Boss Sasha Banks returns,...",Sasha Banks reemerged on the Raw after SummerS...,Appearing on WWE programming for the first tim...,WWE WrestleMania Sasha Banks SummerSlam Nataly...


#Creating new user profile

In [ ]:
#giving some news from data

Now we will create user profile which we will use to in our recommandation system

In [ ]:
data['cat'] = pd.factorize(data['Category'])[0]

def factorizer(data):
  uci = list(data['cat'].unique())
  L=[]
  for i in range(len(uci)):
      L.append(data.groupby(['cat']).get_group(uci[i]))
  for i in L:
    i['sub_cat'] , f = pd.factorize(i['Sub_Category'])
  for i in range(len(L)):
    L[i]['luffy'] = L[i]['cat'].astype(str) + '.' + L[i]['sub_cat'].astype(str)
  return L
data

,Category,Sub_Category,Title,Synopsis,News,tags,cat
0,Sports,Badminton,BWF World Tour Finals: Fighting PV Sindhu lose...,This was PV Sindhu's 16th defeat to Tai Tzu Yi...,World champion shuttler P V Sindhu went down f...,Tai Tzu Ying Taiwan $1.5 million HSBC BWF Worl...,0
1,Sports,Badminton,"World Tour Finals Preview: PV Sindhu, recharge...",With the Indian having played more matches tha...,Carolina Marin (50 total) played 24 tournament...,Carolina Marin PV Sindhu Indian Marin Sindhu O...,0
2,Sports,Badminton,Satwiksairaj’s offence gets neutralised by sav...,Satwiksairaj Rankireddy uses big smash to kill...,One would have to be blind to not figure that ...,Satwiksairaj Rankireddy Neutralising Malaysian...,0
3,Sports,Badminton,Dream run of Indian doubles pairs end with sem...,"Up against the world number three Thai pair, S...",The Indian mixed doubles pair of Satwiksairaj ...,Indian Satwiksairaj Rankireddy Ashwini Ponnapp...,0
4,Sports,Badminton,Satwik-Chirag’s impressive run ends with semif...,The Indian pair had participated in Super 1000...,Tokyo Olympics medal contender Satwiksairaj Ra...,Tokyo Olympics Satwiksairaj Rankireddy Chirag ...,0
...,...,...,...,...,...,...,...
5918,Entertainment,Box-office-collection,Kesari box office collection Day 8: Akshay Kum...,Kesari box office collection Day 8: Akshay Kum...,"Kesari, starring Akshay Kumar in the lead role...",Kesari Akshay Kumar Rs 105.86 crore Taran Adar...,3
5919,Entertainment,Box-office-collection,Junglee box office prediction: Vidyut Jammwal ...,Junglee box office prediction: Junglee will ha...,Vidyut Jammwal’s Junglee has caught the fancy ...,Vidyut Jammwal’s Junglee Kesari Notebook Chuck...,3
5920,Entertainment,Box-office-collection,Luka Chuppi box office collection Day 26: Kart...,Luka Chuppi box office collection Day 26: Krit...,Kriti Sanon and Kartik Aaryan’s film Luka Chup...,Kriti Sanon Kartik Aaryan Luka Chuppi Rs 43 la...,3
5921,Entertainment,Box-office-collection,Badla box office collection day 20: Going gets...,Badla box office collection day 20: Taapsee Pa...,"After more than 2 weeks, Taapsee Pannu’s film ...",Taapsee Pannu’s Badla Badla Rs 1 crore only Rs...,3


factorized daata look like this

In [ ]:
L = factorizer(data)
L

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


[     Category   Sub_Category  ... sub_cat luffy
 0      Sports      Badminton  ...       0   0.0
 1      Sports      Badminton  ...       0   0.0
 2      Sports      Badminton  ...       0   0.0
 3      Sports      Badminton  ...       0   0.0
 4      Sports      Badminton  ...       0   0.0
 ...       ...            ...  ...     ...   ...
 2368   Sports  Wwe-wrestling  ...       6   0.6
 2369   Sports  Wwe-wrestling  ...       6   0.6
 2370   Sports  Wwe-wrestling  ...       6   0.6
 2371   Sports  Wwe-wrestling  ...       6   0.6
 2372   Sports  Wwe-wrestling  ...       6   0.6
 
 [2373 rows x 9 columns],       Category Sub_Category  ... sub_cat luffy
 2373  Business     Aviation  ...       0   1.0
 2374  Business     Aviation  ...       0   1.0
 2375  Business     Aviation  ...       0   1.0
 2376  Business     Aviation  ...       0   1.0
 2377  Business     Aviation  ...       0   1.0
 ...        ...          ...  ...     ...   ...
 4101  Business       Market  ...       4   1.4
 

Hear we are creating first user interface what he will get on his first time visiting 
he will get news from  all the section so for firct time we are giving him selection freedom to select his favorit catagories

In [ ]:
#This is what user will get on his first time

def first_time_user(data , num):
  sug_ls = []
  L = factorizer(data)
  for sec in L:
    sug = sec.sample(n = num)
    sug_ls.append(sug)
  sug_df = pd.concat(sug_ls)
  return sug_df
    


In [ ]:
first_data = first_time_user(data , 5)
first_data = first_data.reset_index()
first_data = first_data.drop(['cat', 'sub_cat','luffy'], axis=1)

first_data


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,index,Category,Sub_Category,Title,Synopsis,News,tags
0,533,Sports,Cricket,"Delhi record second successive win, beat Andhr...",Delhi chased the target with three overs to sp...,Delhi recorded their second straight win in th...,Delhi Andhra Pradesh Elite Group E the Wankhed...
1,637,Sports,Cricket,Delhi Capitals appoint Pravin Amre as assistan...,"Pravin Amre, who earlier served as the franchi...",Indian Premier League (IPL) franchise Delhi Ca...,Indian IPL Delhi Capitals Pravin Amre Amre Del...
2,2055,Sports,Wwe-wrestling,"WWE Hell in a Cell 2020 Live Streaming, Date a...","WWE Hell in a Cell 2020 Live Streaming, Date a...","WWE Hell in a Cell 2020 Live Streaming, Date a...",Universal Championship Jey Uso Cell Jimmy Uso ...
3,1766,Sports,Tennis,Stefanos Tsitsipas reveals injury scare ahead ...,Stefanos Tsitsipas said he had picked up a leg...,Stefanos Tsitsipas says he is unsure how his A...,Stefanos Tsitsipas Greek the Paris Masters ATP...
4,199,Sports,Badminton,"World conqueror PV Sindhu, fast-rising Lakshya...","In doubles, Satwiksairaj Rankireddy and Chirag...",A momentous gold at the world championships mo...,the world championships Lakshya Sen Indian 201...
5,2902,Business,Banking-and-finance,Falling cash usage in May-Jul: Transaction val...,"During May, June, and July, for example, the t...",Mounting job losses and salary cuts across sec...,the National Payments Corporation of India NPC...
6,4031,Business,Market,"Sebi shortlists Bharti Airtel, Wipro, others t...",The other shortlisted companies are -- Hewlett...,Markets regulator Sebi has shortlisted eight c...,Sebi Bharti Airtel Wipro Tata Communications H...
7,3391,Business,Companies,Bajaj Auto posts 9 per cent fall in total sale...,"Total domestic sales in August stood at 1,85,8...",Bajaj Auto on Wednesday reported a 9 per cent ...,"Bajaj Auto 9 per cent Pune 1,85,879 11 per cen..."
8,2865,Business,Banking-and-finance,"SC protect accounts, which were not declared N...",The apex court passed the order while noting t...,The Supreme Court Thursday directed that accou...,The Supreme Court NPA Ashok Bhushan The apex c...
9,3354,Business,Companies,Toyota Kirloskar Motor reiterates its commitme...,Toyota Kirloskar Motor (TKM) is a joint ventur...,Toyota Kirloskar Motor (TKM) on Thursday said ...,Toyota Kirloskar Motor TKM Indian India Innova...


We have 5-5 news form each section so user will select 5 news 

In [ ]:
def user_selection(first_data):
  in_idx = []
  for i in range(0 ,5):
    inp = int(input('input index no of the news  : '))
    in_idx.append(inp)
    idx_ls = []
    for idx in in_idx:
      idx_d = idx_to_df(idx , first_data)
      idx_ls.append(idx_d)
    idx_df = pd.concat(idx_ls)

  return idx_df
user_df = user_selection(first_data)




input index no of the news  : 0
input index no of the news  : 1
input index no of the news  : 2
input index no of the news  : 17
input index no of the news  : 18


as above we've created the user profile for the first user which will give some recommandations

And as we can seeeee've selected some sports news and Entertainment	news and we are getting recommandations from the same secton so our recommandation system is working well. Even in sports section we've selected crickt and Wwe-wrestling	 sub section and as we can see we are getting recommandations from the same section wonderful....

In [ ]:
recommander(user_df['tags'] , 10)

It took 0.005433082580566406 seconds to query forest.


,Category,Sub_Category,Title,Synopsis,News,tags,cat
5792,Entertainment,Box-office-collection,War box office collection Day 15: Hrithik-Tige...,War box office collection Day 15: The YRF acti...,Hrithik Roshan and Tiger Shroff starrer War is...,Hrithik Roshan Tiger Shroff Rs 300 crore Rs 53...,3
2055,Sports,Wwe-wrestling,"WWE Hell in a Cell 2020 Live Streaming, Date a...","WWE Hell in a Cell 2020 Live Streaming, Date a...","WWE Hell in a Cell 2020 Live Streaming, Date a...",Universal Championship Jey Uso Cell Jimmy Uso ...,0
649,Sports,Cricket,Navdeep will rise to the occasion if handed a ...,"Does the scrawny pacer from Taraori, Haryana, ...","Navdeep Saini is yet to make his Test debut, e...",Navdeep Saini India Umesh Yadav Saini Sydney T...,0
5802,Entertainment,Box-office-collection,War box office prediction: Hrithik Roshan and ...,"Releasing in over 4000 screens, Hrithik Roshan...","The Gandhi Jayanti release of the year, War, s...",Gandhi Jayanti War Hrithik Roshan Tiger Shroff...,3
431,Sports,Cricket,Rajasthan Royals release Steve Smith; Chennai ...,Rajasthan Royals have also appointed former Sr...,The IPL player retention/release day doesn’t o...,IPL IPL IPL Samson RoyalsRajasthan Royals Stev...,0
3281,Business,Companies,"Cox & Kings loaned out Rs 6,071 crore to at le...",The travel firm came under the lens after the ...,"A forensic audit of Cox & Kings Ltd, the liste...",Cox & Kings Ltd the Enforcement Directorate th...,1
533,Sports,Cricket,"Delhi record second successive win, beat Andhr...",Delhi chased the target with three overs to sp...,Delhi recorded their second straight win in th...,Delhi Andhra Pradesh Elite Group E the Wankhed...,0
2070,Sports,Wwe-wrestling,Triple H: Covid-19 has made this time frame ‘i...,Triple H opens up about the difficulties WWE f...,World Wrestling Federation (WWE) has managed t...,World Wrestling Federation WWE WWE Paul Michae...,0
2071,Sports,Wwe-wrestling,"Slams, not dunks: WWE replaces NBA at one Flor...",Pro wrestling is replacing pro basketball at t...,Pro wrestling is replacing pro basketball at t...,Florida WWE the Amway Center Orlando NBA The O...,0
606,Sports,Cricket,"Syed Mushtaq Ali T20 Trophy 2020/21: Schedule,...",Syed Mushtaq Ali T20 Trophy 2020/21: All you n...,Domestic cricket finally returns to India afte...,India Ranji Trophy Indian Mushtaq Ali T20 Trop...,0


Now we will create a profile for the old user 

In [ ]:
def old_user(data ,num):
  selection = first_time_user(data , num)
  selection = selection.reset_index()
  selection = selection.drop(['cat', 'sub_cat','luffy'], axis=1)
  user_data = selection[['index', 'tags']]
  return user_data





We have generated this clickstream data in saparate ipynb file that we've attached

In [ ]:
cs = pd.read_csv('/content/gdrive/MyDrive/StrawHats projects/StrawHats_clickstream.csv')
cs = cs.drop(['Unnamed: 0','served_section','Catagory','Sub_Catagory'], axis=1)
cs

,Click,UserID,Time_Spent
0,0,1.0,0.000000
1,1,1.0,1.764861
2,1,1.0,0.635004
3,1,1.0,0.000000
4,0,1.0,0.000000
...,...,...,...
149995,1,100.0,8.806663
149996,1,100.0,11.005164
149997,1,100.0,10.909229
149998,1,100.0,17.250826


Hear we are taking just one user to represent so we are taking following notes


1) our user's intrusts so user will spnt their most of the time in their intrest section


2) Not repetitive recommandations


In [ ]:
user_selection = old_user(data, 20)
user_1 = cs[cs['UserID'] == 1.0].sample( n = 80)
user_1 = user_1.reset_index()
user_1 = user_1.drop(['index'],axis=1)
user_1 = user_1.join(user_selection)
user_1.sort_values("Time_Spent", axis = 0, ascending = False,
                 inplace = True, na_position ='first')
user_1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Click,UserID,Time_Spent,index,tags
2,1,1.0,17.250826,1002,India Hardik Pandya ODI Australia Virat Kohli ...
27,1,1.0,17.250826,3060,the Reserve Bank of India HDFC Bank overdues H...
14,1,1.0,17.250826,1822,Serena Williams the French Open Kristie Ahn Wi...
49,1,1.0,17.250826,5556,OnePlus Pete Lau Weibo OnePlus 8 OnePlus Rs 42...
24,1,1.0,17.250826,3507,"The Finance Ministry Rs 9,879.61 Ministry Tami..."
...,...,...,...,...,...
9,0,1.0,0.000000,1457,Lewis Hamilton Briton George Floyd Minneapolis...
54,1,1.0,0.000000,4193,Amazfit Neo Amazfit Neo Caravaan India Rs The ...
43,0,1.0,0.000000,4929,Vivo Vivo four-pixel Vivo V17 Pro’s USP Macro ...
17,1,1.0,0.000000,924,Kerala Blasters’ FC Goa the Indian Super Leagu...


Lets test

In [ ]:
user_1['tags'][:5]

2     India Hardik Pandya ODI Australia Virat Kohli ...
27    the Reserve Bank of India HDFC Bank overdues H...
14    Serena Williams the French Open Kristie Ahn Wi...
49    OnePlus Pete Lau Weibo OnePlus 8 OnePlus Rs 42...
24    The Finance Ministry Rs 9,879.61 Ministry Tami...
Name: tags, dtype: object

In [ ]:
recommander(user_1['tags'][:5] , 10)

It took 0.004212379455566406 seconds to query forest.


,Category,Sub_Category,Title,Synopsis,News,tags,cat
963,Sports,Football,Champions League wrap: Liverpool and Porto adv...,Zinedine Zidane’s side would have advanced wit...,While former champions Liverpool and Porto eas...,Liverpool Porto the Champions League Real Madr...,0
677,Sports,Football,Thomas Tuchel appointed Chelsea manager follow...,"Thomas Tuchel, the former Borussia Dortmund an...",Thomas Tuchel became the latest manager to tak...,Thomas Tuchel Chelsea Premier League German Fr...,0
5223,Technology,Science,"Future warming arriving earlier than expected,...",Dozens of extreme events have occurred along t...,Climate scientists have previously warned that...,Science Advances The emergence of heat and hum...,2
1610,Sports,Motor-sport,Azerbaijan GP: Charles Leclerc leads Ferrari p...,Charles Leclerc set a scorching pace on an inc...,Charles Leclerc set a scorching pace on an inc...,Charles Leclerc Azerbaijan Grand Prix Ferrari ...,0
3409,Business,Companies,"Ducati launches all-new Panigale V2 in India, ...",The superbike is powered by a 955-cc BS-VI eng...,Italian superbike maker Ducati on Wednesday sa...,Italian Ducati Panigale India Rs 16.99 lakh BS...,1
2289,Sports,Wwe-wrestling,WWE Raw: Kurt Angle announces his WrestleMania...,WWE Raw witnessed Kurt Angle finally opening t...,Kurt Angle has an opponent for his final match...,Kurt Angle John Cena Shelton Benjamin the WWE ...,0
1395,Sports,Motor-sport,Mercedes F1 team member tests positive for COV...,Formula 1 last week reported 10 positive COVID...,Formula One champions Mercedes reported on Thu...,Mercedes Lewis Hamilton Valtteri Bottas Britis...,0
1175,Sports,Hockey,"India men to face Russia, women draw USA in fi...",Eight-time Olympic champions India men are ran...,"In a favourable draw, India’s men’s hockey tea...",India Russia the Tokyo Olympics Russian Russia...,0
1720,Sports,Tennis,Tennis in 2020: Novak Djokovic’s mixed year to...,"The Tennis Yearender: The covid interruption, ...",Novak Djokovic remained the highlight of tenni...,Novak Djokovic Australian Serbian DjokovicDjok...,0
1822,Sports,Tennis,French Open 2020: Serena Williams reaches seco...,Serena Williams is a three-time French Open ch...,Serena Williams advanced to the second round o...,Serena Williams the French Open Kristie Ahn Wi...,0


In [ ]:
def user_recommandaton(user_data_tags):
    re_1 = user_data_tags[:2]
    re_2 = user_data_tags[2:4]
    re_3 = user_data_tags[4:6]
    re_4 = user_data_tags[6:8]
    re_5 = user_data_tags[8:10]
    re = [re_1, re_2, re_3, re_4, re_5]
    recommandations = []
    for i in re:
      user_reco = recommander(i , 2)
      recommandations.append(user_reco)
    recommandations = pd.concat(recommandations)
    return recommandations
user_recommandaton(user_1['tags'])




It took 0.003020763397216797 seconds to query forest.
It took 0.003388643264770508 seconds to query forest.
It took 0.003313779830932617 seconds to query forest.
It took 0.003461122512817383 seconds to query forest.
It took 0.0029435157775878906 seconds to query forest.


,Category,Sub_Category,Title,Synopsis,News,tags,cat
2976,Business,Banking-and-finance,RBI proposes to limit tenures of CEOs & whole-...,The RBI has said it is desirable to limit the ...,The Reserve Bank of India has proposed to rest...,The Reserve Bank of India RBI WTD the Departme...,1
3049,Business,Banking-and-finance,Reserve Bank puts on hold two key HDFC Bank ap...,"The RBI move, which came over four months afte...",The Reserve Bank of India (RBI) has put on hol...,The Reserve Bank of India Sashidhar Jagdishan ...,1
1540,Sports,Motor-sport,"Formula 1: Valtteri Bottas takes US pole, Lewi...",Valtteri Bottas will need a victory on Sunday ...,Mercedes’ Valtteri Bottas snatched pole positi...,Mercedes Valtteri Bottas the U.S. Grand Prix L...,0
1822,Sports,Tennis,French Open 2020: Serena Williams reaches seco...,Serena Williams is a three-time French Open ch...,Serena Williams advanced to the second round o...,Serena Williams the French Open Kristie Ahn Wi...,0
2131,Sports,Wwe-wrestling,"On Stone Cold Steve Austin Day, Coronavirus cr...",Stone Cold Steve Austin asked for a hell yeah ...,Stone Cold Steve Austin has been one of the mo...,Steve Austin WWE the Royal Rumble Austin 3:16”...,0
5755,Entertainment,Box-office-collection,Dabangg 3 box office collection prediction: Sa...,Considering the box office success of the last...,Chulbul Robinhood Pandey aka Salman Khan is re...,Chulbul Robinhood Pandey Salman Khan Dabangg D...,3
5596,Technology,Techook,"The best smart speakers under Rs 20,000 in 2020",If you are looking for a smart speaker to cont...,Smart speakers are in the rage these days as m...,Bluetooth Apple HomePod Apple iOS Apple HomeKi...,2
4573,Technology,Laptops,"Acer Swift 7 with thin bezels, compact design ...",CES 2019: Acer's Swift 7 features a high-resol...,"At CES 2019, Acer has launched the Swift 7 not...",CES 2019 Acer Swift 7 92 per cent Intel Core 9...,2
4112,Technology,Gadgets,LG’s new Ultrafine Ergo 4K monitor can be swiv...,The LG Ultrafine Ergo monitor can be tilted in...,LG has launched its new Ultrafine Display Erg...,LG Ultrafine Display Ergo India The LG 32UN880...,2
2695,Business,Aviation,Air India needs to survive till it is sold: CM...,Civil Aviation Minister Hardeep Singh Puri had...,As the central government is planning to invit...,Air India CMD Facebook Air India Ashwani Lohan...,1


So hear we are getting all the recommandations for one old user and we are getting preety good recommandations for old user also.

Similarly we can give recommandations to all the users

In [51]:
user_id = cs['UserID'].unique()
user_id = user_id.tolist()
user = []
for i in user_id:
  user_selection = old_user(data, 20)
  user_1 = cs[cs['UserID'] == i].sample( n = 80)
  user_1 = user_1.reset_index()
  user_1 = user_1.drop(['index'],axis=1)
  user_1 = user_1.join(user_selection)
  user_1.sort_values("Time_Spent", axis = 0, ascending = False,
                  inplace = True, na_position ='first')
  user.append(user_1)
rec_for_all = []
for L in user:
  L = L['tags']
  rec = user_recommandaton(L)
  rec_for_all.append(rec)
rec_for_all


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

It took 0.006429433822631836 seconds to query forest.
It took 0.003559589385986328 seconds to query forest.
It took 0.0045359134674072266 seconds to query forest.
It took 0.0035538673400878906 seconds to query forest.
It took 0.003489255905151367 seconds to query forest.
It took 0.003573894500732422 seconds to query forest.
It took 0.003138303756713867 seconds to query forest.
It took 0.003746509552001953 seconds to query forest.
It took 0.0025606155395507812 seconds to query forest.
It took 0.003406047821044922 seconds to query forest.
It took 0.0029611587524414062 seconds to query forest.
It took 0.0031583309173583984 seconds to query forest.
It took 0.003000497817993164 seconds to query forest.
It took 0.004415035247802734 seconds to query forest.
It took 0.003400564193725586 seconds to query forest.
It took 0.0035936832427978516 seconds to query forest.
It took 0.00321197509765625 seconds to query forest.
It took 0.0029888153076171875 seconds to query forest.
It took 0.003175258636

[           Category  ... cat
 5355     Technology  ...   2
 4212     Technology  ...   2
 5801  Entertainment  ...   3
 5805  Entertainment  ...   3
 448          Sports  ...   0
 1664         Sports  ...   0
 5393     Technology  ...   2
 5629     Technology  ...   2
 5733  Entertainment  ...   3
 5735  Entertainment  ...   3
 
 [10 rows x 7 columns],            Category  ... cat
 1297         Sports  ...   0
 2131         Sports  ...   0
 5789  Entertainment  ...   3
 5799  Entertainment  ...   3
 5830  Entertainment  ...   3
 5279     Technology  ...   2
 2961       Business  ...   1
 2996       Business  ...   1
 5081     Technology  ...   2
 5175     Technology  ...   2
 
 [10 rows x 7 columns],            Category  ... cat
 5811  Entertainment  ...   3
 5839  Entertainment  ...   3
 3730       Business  ...   1
 4397     Technology  ...   2
 5867  Entertainment  ...   3
 5878  Entertainment  ...   3
 5865  Entertainment  ...   3
 5686  Entertainment  ...   3
 347          Sports

So hear we have recommandations for every user on the server Have a Good Day...

#Summery
We have used two powerful algorithms to devlop our recommandation system NER+LSH

As we can see the results we are getting better recommandations by using these two algorithms even these model is preety fast and handy to use even for all the 100 users with 150000 data points this algorithm gives recommandations to each user in less then one min which is extreemly fast.. 

So even for the small servers we can use this recommandation system to save time and this model will work efficientely.

only slow process in this algorithm is getting NER tags on normal CPU it will take too much time but hear we are using google collab GPUs so we are getting results within 10 min but once NER tagging is done everything is blezzing fast.

For collabrative filturing we've sticked to our algorithm and decided not to use any other mathodes because mathamatically speaking every other mathod will take more computational time then this model and even if we are using K-nearest nabour mathod then we will definetly loose more data then this mathod even for metrix factorization we will make a model by using just stastictical data but in this mathod we are using news tags by using NER so we will not loose any details about users intrusts.

Hope you will like our algorithm for Jhakkas Newswala. download this notebook and run into google collab and experience this amazing algorithm.

Thankyou..